In [57]:
import requests
import pandas as pd
import numpy as np
import io
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder
import matplotlib.pyplot as plt

In [58]:
gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable='2021-22',
league_id_nullable='00',
season_type_nullable='Regular Season')
games = gamefinder.get_data_frames()[0]

game_ids = games['GAME_ID'].unique().tolist()

def get_data(game_id):
    play_by_play_url = "https://cdn.nba.com/static/json/liveData/playbyplay/playbyplay_"+game_id+".json"
    response = requests.get(url=play_by_play_url).json()
    play_by_play = response['game']['actions']
    df = pd.DataFrame(play_by_play)
    df['gameid'] = game_id
    return df

In [59]:
game_data_agg = []
for game_id in game_ids:
    game_data_agg.append(get_data(game_id))

In [60]:
jump_counter = 0
first_counter = 0

for game_data in game_data_agg:
    jump_won_team = game_data.loc[1, "teamTricode"]
    found = False
    i=2
    home_away = {}
    while (not found):
        if game_data.loc[i, "shotResult"] == "Made":
            if game_data.loc[i, "scoreHome"] != 0:
                home_away['home'] = game_data.at[i, "teamTricode"]
                home_away['away'] = None
            else:
                home_away['away'] = game_data.at[i, "teamTricode"]
                home_away['home'] = None
            first_scored = game_data.loc[i, "teamTricode"]
            found = True
        i = i+1
    home_won = (int(game_data["scoreHome"].iloc[-1]) > int(game_data["scoreAway"].iloc[-1]))
    if (home_won and home_away['home'] == jump_won_team) or (not home_won and home_away['away'] == jump_won_team):
        jump_counter = jump_counter + 1
    if (home_won and home_away['home'] == first_scored) or (not home_won and home_away['away'] == first_scored):
        first_counter = first_counter + 1

jump_win_pct = jump_counter/len(game_data_agg)
print(jump_win_pct)
first_win_pct = first_counter/len(game_data_agg)
print(first_win_pct)

0.35833333333333334
0.5435185185185185


In [61]:
counter = 0
total = 0
games = []

for game_data in game_data_agg:
    scores = game_data.loc[(game_data['actionType'] == 'period') & (game_data['subType'] == 'start'), ['scoreHome', 'scoreAway']]
    home_score = int(scores['scoreHome'].iloc[1])
    away_score = int(scores['scoreAway'].iloc[1])
    if home_score > (away_score):
        home_lead = True
    elif away_score > (home_score):
        home_lead = False

    else:
        continue
    
    home_won = (int(game_data["scoreHome"].iloc[-1]) > int(game_data["scoreAway"].iloc[-1]))
    if (home_lead and home_won) or ((not home_lead) and (not home_won)):
        counter = counter + 1
    total = total + 1

lead_win_pct = counter/total
print(counter)
print(lead_win_pct)

689
0.6708860759493671
